In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd, tweepy
%matplotlib inline
%config IPCompleter.greedy=True
pd.set_option('display.max_colwidth', None)
#References https://github.com/JustAnotherArchivist/snscrape
#           https://github.com/cedoard/snscrape_twitter
#           https://medium.com/@jcldinco/downloading-historical-tweets-using-tweet-ids-via-snscrape-and-tweepy-5f4ecbf19032


In [ ]:
consumer_key = '4AusmeRMGGK6yGuPHEkO9RgKi'
consumer_secret = 'ND3wTwuNICtEBbHBlpbYOj1C7ubkWnA8j9T6RNFUI6QRqyMOwo'
access_token = '767125259258568704-91nLEbtiwA5GOlGvXVW6vVas0ntERuF'
access_token_secret = 'twTMHB18q8R93w89NsOOfeBJfovXVITlNmIDH63VZbY0h'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
tweet_url = pd.read_csv("months_seperate_2020/brexit_dec2020.txt", index_col= None, header = None, names = ["links"])
tweet_url.head()

In [ ]:
af = lambda x: x["links"].split("/")[-1]
tweet_url['id'] = tweet_url.apply(af, axis=1)

In [ ]:
ids = tweet_url['id'].tolist()

In [ ]:
tweet_url.head()

In [ ]:
total_count = len(ids)
chunks = (total_count - 1) // 50 + 1

In [ ]:
def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"TweetID": status.id,
                     "Content":status.full_text,
                     "Location":status.user.location,
                     "Username":status.user.screen_name,
                     "Created At":status.created_at,
                     "Retweet-Count":status.retweet_count,
                     "Favorites":status.favorite_count
                     }
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv("csv_months_seperate_2020/tweets_raw_dec2020.csv", mode="a")

In [ ]:
for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result = fetch_tw(batch)